# Binary Sentiment Analysis on IMDB Reviews Dataset

# Libraries & Modules

In [1]:
# Manage Warnings
import warnings
warnings.filterwarnings("ignore")

# Libraris & Modules
import os
import sys
import shutil
import re as regex
import matplotlib.pyplot as plt
from string import punctuation

# TensorFlow Log Level
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# TensorFlow Libraries
import tensorflow as tf

# Environment/Versions
print("Python Env : ", sys.version)
print("TensorFlow : ", tf.__version__)

# Random Generator Seed
random_seed = 47

Python Env :  3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]
TensorFlow :  2.12.0


# Downloading & Exploring Dataset

In [2]:
# Dataset Download Path
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

# Retrieve File (and untar)
dataset = tf.keras.utils.get_file(fname="aclImdb_v1", 
                                  origin=url, 
                                  untar=True, 
                                  cache_dir=".", 
                                  cache_subdir="")

# Dataset Directory
dataset_dir = os.path.join(os.path.dirname(dataset), "aclImdb")

84125825/84125825 [==============================] - 9s 0us/step


In [3]:
# List Dataset Sub-Directories & Files
display(os.listdir(dataset_dir))

['imdb.vocab', 'README', 'train', 'test', 'imdbEr.txt']

In [4]:
# Review Readme File
with open(os.path.join(dataset_dir, 'README'), 'r') as f:
    print(f.read())

Large Movie Review Dataset v1.0

Overview

This dataset contains movie reviews along with their associated binary
sentiment polarity labels. It is intended to serve as a benchmark for
sentiment classification. This document outlines how the dataset was
gathered, and how to use the files provided. 

Dataset 

The core dataset contains 50,000 reviews split evenly into 25k train
and 25k test sets. The overall distribution of labels is balanced (25k
pos and 25k neg). We also include an additional 50,000 unlabeled
documents for unsupervised learning. 

In the entire collection, no more than 30 reviews are allowed for any
given movie because reviews for the same movie tend to have correlated
ratings. Further, the train and test sets contain a disjoint set of
movies, so no significant performance is obtained by memorizing
movie-unique terms and their associated with observed labels.  In the
labeled train/test sets, a negative review has a score <= 4 out of 10,
and a positive review has a scor

In [5]:
# Train & Test Directories
display(os.listdir(f"{dataset_dir}/train"))
display(os.listdir(f"{dataset_dir}/test"))

['neg',
 'urls_neg.txt',
 'unsup',
 'unsupBow.feat',
 'urls_pos.txt',
 'pos',
 'labeledBow.feat',
 'urls_unsup.txt']

['neg', 'urls_neg.txt', 'urls_pos.txt', 'pos', 'labeledBow.feat']

In [6]:
# Training Data Directory
train_dir = os.path.join(dataset_dir, "train")
train_dir_items = os.listdir(train_dir)
train_dir_items

['neg',
 'urls_neg.txt',
 'unsup',
 'unsupBow.feat',
 'urls_pos.txt',
 'pos',
 'labeledBow.feat',
 'urls_unsup.txt']

In [7]:
# Testing Data Directory
test_dir = os.path.join(dataset_dir, "test")
os.listdir(test_dir)

['neg', 'urls_neg.txt', 'urls_pos.txt', 'pos', 'labeledBow.feat']

In [8]:
# View a sample positive review.
train_files = os.listdir(os.path.join(train_dir, "pos"))
sample_fname = train_files[2]
print(f"File : {sample_fname}\n")
with open(os.path.join(train_dir, f"pos/{sample_fname}")) as f:
    print(f.read())

File : 1223_7.txt

about a year and a half ago my dad told me about The French Doors. i thought it sounded interesting enough but i didn't try to find it anywhere. Then about a year ago i remembered that film and thought "hey why not" and tried finding it on the internet. eventually after about a week of looking i found it on atom films. i called my dad over to the computer and said to him" hey dad I've found that creepy film you told me about ages ago!" He smiled at me, turned round turned off the lights so it was pitch black apart from the computer screen and told me to watch it. I started off fine...Then when he started getting worried about whatever was there i found it very unnerving. at the end i pushed back my chair and stood up...it made me jump!:P if you haven't seen this film i highly recommend you do because it is well worth it. even after the fourth or fifth time its still unsettling.<br /><br />GREAT FILM!


In [9]:
tf.keras.utils.text_dataset_from_directory?

In [10]:
# Prepare Training Directory as per 'text_dataset_from_directory' Requirements.
# -----------------------------------------------------------------------------
# Create New Directory for Unsupervised Data
train_unsup_dir = os.path.join(dataset_dir, "train_unsup")
if not os.path.exists(train_unsup_dir):
    os.makedirs(train_unsup_dir)
else:
    pass
display(os.listdir(dataset_dir))
# Collect Unsupervised Data Files and Directories
file_list = [_ for _ in train_dir_items if "unsup" in _]
# Source & Destination Directories
source_dir = train_dir
destination_dir = train_unsup_dir
# Move Files and Directories
for file in file_list:
    source_file = os.path.join(source_dir, file)
    destination_file = os.path.join(destination_dir, file)
    # Move as per Directory/File
    if os.path.isdir(source_file):
        shutil.copytree(source_file, destination_file)
        shutil.rmtree(source_file)
    else:
        shutil.move(source_file, destination_file)

# Review Training & Unsupervised Training Data Directories
display(os.listdir(train_dir))
display(os.listdir(train_unsup_dir))

['imdb.vocab', 'README', 'train', 'test', 'imdbEr.txt', 'train_unsup']

['neg', 'urls_neg.txt', 'urls_pos.txt', 'pos', 'labeledBow.feat']

['unsup', 'unsupBow.feat', 'urls_unsup.txt']

# Training, Validation & Testing Datasets

In [11]:
# Training Dataset
print("Training Dataset :")
train_ds = tf.keras.utils.text_dataset_from_directory(directory=train_dir, 
                                                      batch_size=8, 
                                                      validation_split=0.2, 
                                                      subset='training', 
                                                      seed=random_seed)

# Validation Dataset
print("\n")
print("Validation Dataset :")
val_ds = tf.keras.utils.text_dataset_from_directory(directory=train_dir, 
                                                    batch_size=8, 
                                                    validation_split=0.2, 
                                                    subset='validation', 
                                                    seed=random_seed)

# Testing Dataset
print("\n")
print("Testing Dataset :")
test_ds = tf.keras.utils.text_dataset_from_directory(directory=test_dir, 
                                                     batch_size=8)

Training Dataset :
Found 25000 files belonging to 2 classes.
Using 20000 files for training.


Validation Dataset :
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


Testing Dataset :
Found 25000 files belonging to 2 classes.


In [12]:
# Review Dataset
for x, y in train_ds.take(1):
    print(f"Dataset Element 1 (Labels)  : {tf.shape(y)}")
    print(f"Dataset Element 1 (Reviews) : {tf.shape(x)}\n")
    for i in range(5):
        tf.print(f"LABEL  : {y[i]}")
        tf.print(f"REVIEW : {x[i]}")
        tf.print("-"*80)

Dataset Element 1 (Labels)  : [8]
Dataset Element 1 (Reviews) : [8]

LABEL  : 0
REVIEW : b"I wouldn't say this is a *bad* movie. Unfortunately for me, I get the feeling that the more you know about fencing, the worse it gets simply due to the fact that it becomes totally unrealistic. I've been fencing since i was 14 years old, and this movie portrays it very poorly. F. Murray Abraham is good (and appears to have some fencing background), but most of the other actors--especially the students--just seem to be lost."
--------------------------------------------------------------------------------
LABEL  : 1
REVIEW : b'Family problems abound in real life and that is what this movie is about. Love can hold the members together through out the ordeals and trials and that is what this movie is about. One man, Daddy, has the maturity and fortitude to sustain the family in the face of adversity. The kids grow up,one all be it, in the hard way, to realize that no matter how old they or a parent 

In [13]:
# Class Names Corresponding to Index (as Numerical Label)
display(train_ds.class_names)

['neg', 'pos']

# Data Preparation

## Custom Standardization Function

Removing Punctuations & HTML Elements.

In [14]:
# Function Definition : Remove HTML Tags & Punctuations
def custom_standardize(input_data):
    """Remove HTML Tags & Punctuations"""
    
    # Convert to Lowercase
    lowercase = tf.strings.lower(input_data)
    
    # Remove the '<br />' Tag
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    
    # Remove Punctuations
    stripped_html_punc = tf.strings.regex_replace(stripped_html,
        "[%s]" % regex.escape(punctuation), ""
    )
    
    return stripped_html_punc

## Tokenization & Vectorization

- Custom function for Standardization
- Splitting strings into tokens. --> tf.keras.layers.TextVectorization()
- Converting tokens into numbers for neural network.

In [15]:
# Calculate Maximum Features Value
# --------------------------------
# Collect all Word Tokens provided in Dataset
vocab_tokens = list()
with open(os.path.join(dataset_dir, "imdb.vocab"), "r") as f:
    for line in f:
        vocab_tokens.append(line.strip())
vocab_tokens = set(vocab_tokens)       

# Calc. Max. Features rounded to next hundred
max_features = round(len(vocab_tokens)+100, -2)

# Delete Token Object
del vocab_tokens

# Display Max. Features
print("Maximum Features(Tokens) in Entire Dataset : ", max_features)

# Set Maximum Sequence Length
# ---------------------------
max_seq_len = 250
print("Maximum Output Sequence Length of Vectorization Layer : ", max_seq_len)

Maximum Features(Tokens) in Entire Dataset :  89600
Maximum Output Sequence Length of Vectorization Layer :  250


In [16]:
# Vectorization Layer
T_Vect = tf.keras.layers.TextVectorization(standardize=custom_standardize, 
                                           max_tokens=max_features, 
                                           output_mode="int", 
                                           output_sequence_length=max_seq_len)
display(T_Vect)

In [17]:
# Fit State of Preprocessing Layer to the Dataset
training_text = train_ds.map(lambda x, _ : x)
T_Vect.adapt(training_text)
display(T_Vect)

In [18]:
# Review Vectorization Layer's Output
for x, y in train_ds.take(1):
    print(f"Dataset Element 1 (Labels)  : {tf.shape(y)}")
    print(f"Dataset Element 1 (Reviews) : {tf.shape(x)}\n")
    for i in range(1):
        tf.print(f"LABEL  : {y[i]}")
        tf.print(f"REVIEW : {x[i]}")
        tf.print("-"*80)
        tf.print(f"VECTORIZED REVIEW :")
        vectorized = T_Vect(tf.expand_dims(x[i], -1))
        print(vectorized)
        tf.print("-"*80)
        print("First Token : ", vectorized[0][0])
        print("Corresponding Vocabulary : ", T_Vect.get_vocabulary()[vectorized[0][0]])
        print("Vocabulary Size : ", len(T_Vect.get_vocabulary()))

Dataset Element 1 (Labels)  : [8]
Dataset Element 1 (Reviews) : [8]

LABEL  : 0
REVIEW : b"This film turned up on local TV here in South Africa recently and I thought that I'd warn even those who enjoy watching B grade bad movies (which I do)that this is not even amusing. The plot concerns a couple visiting a house in the country. Some strangers appear and .... The problem is that most of the film, obviously shot in the early seventies, consists of extreme wide shots of people walking, in real time and awfully slowly, from A to B. This makes the film tedious in the extreme and the expected blood and gore payoff just never happens. I am really curious - how many people have actually watched this from beginning to end?"
--------------------------------------------------------------------------------
VECTORIZED REVIEW :
tf.Tensor(
[[   11    19   695    54    20   686   246   128     8  1207  2334   998
      3    10   197    12   441  2792    53   143    35   350   146   959
   1467    7

# Finalizing Datasets

In [19]:
# Function to Map Vectorization to Dataset
def vectorize(inputs, labels):
    return T_Vect(inputs), labels

In [20]:
# Prepare Training, Testing and Validation Datasets
train_ds = train_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE).map(vectorize)
test_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE).map(vectorize)
val_ds = val_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE).map(vectorize)

# Model Building

In [21]:
# Token Embedding Depth
embedding_dim = 16

# Define Model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(max_features + 1, embedding_dim),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)],
    name="Sequential_Text_Classification"
)

# Compile Model
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=tf.metrics.BinaryAccuracy(threshold=0.0))

model.summary()

Model: "Sequential_Text_Classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 16)          1433616   
                                                                 
 dropout (Dropout)           (None, None, 16)          0         
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
Total params: 1,433,633
Trainable params: 1,433,633
Non-trainable params: 0
__________________________

# Training

In [22]:
hist = model.fit(train_ds, validation_data=val_ds, epochs=10)

Epoch 1/10
2500/2500 [==============================] - 34s 13ms/step - loss: 0.5877 - binary_accuracy: 0.7564 - val_loss: 0.4621 - val_binary_accuracy: 0.8222
Epoch 2/10
2500/2500 [==============================] - 33s 13ms/step - loss: 0.3713 - binary_accuracy: 0.8661 - val_loss: 0.3503 - val_binary_accuracy: 0.8560
Epoch 3/10
2500/2500 [==============================] - 33s 13ms/step - loss: 0.2767 - binary_accuracy: 0.8998 - val_loss: 0.3121 - val_binary_accuracy: 0.8736
Epoch 4/10
2500/2500 [==============================] - 32s 13ms/step - loss: 0.2221 - binary_accuracy: 0.9214 - val_loss: 0.2963 - val_binary_accuracy: 0.8824
Epoch 5/10
2500/2500 [==============================] - 33s 13ms/step - loss: 0.1830 - binary_accuracy: 0.9360 - val_loss: 0.2903 - val_binary_accuracy: 0.8830
Epoch 6/10
2500/2500 [==============================] - 32s 13ms/step - loss: 0.1520 - binary_accuracy: 0.9512 - val_loss: 0.2922 - val_binary_accuracy: 0.8806
Epoch 7/10
2500/2500 [==================

# Evaluation

In [23]:
loss, accuracy = model.evaluate(test_ds)

print(f"Loss     : {round(loss, 2)}")
print(f"Accuracy : {round(accuracy*100, 2)} %")

3125/3125 [==============================] - 6s 2ms/step - loss: 0.3610 - binary_accuracy: 0.8659
Loss     : 0.36
Accuracy : 86.59 %
